In [2]:
import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.hotpotqa import HotPotQA
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

# Set up llama3 with a VLLM client, served on four GPUs. Please note that these URLs will not work for you; you'd need to refer to the documentation to set up your own VLLM/SGLANG server(s).
model="llama3"
lm = dspy.OllamaLocal(model=model, max_tokens=500)
colbert = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.configure(lm=lm, rm=colbert)

/opt/conda/envs/funes-pixi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load data
#train_size=200, dev_size=300
train_size=200
dev_size=11
dataset = HotPotQA(train_seed=1, train_size=train_size, eval_seed=2023, dev_size=dev_size, test_size=0)
trainset = [x.with_inputs('question') for x in dataset.train[0:150]]
valset = [x.with_inputs('question') for x in dataset.train[150:200]]
devset = [x.with_inputs('question') for x in dataset.dev[0:10]]

# show an example datapoint; it's just a question-answer pair
trainset[0]


/opt/conda/envs/funes-pixi/lib/python3.12/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Q: Are both Cangzhou and Qionghai in the Hebei province of China?
A: no


In [4]:

# Question
question=dataset.dev[10]['question']
answer=dataset.dev[10]['answer']
print(f"Q: {question}\nA: {answer}")


Q: What year did the mountain known in Italian as "Monte Vesuvio", erupt?
A: 79 AD


In [8]:
devset

[Example({'question': 'Are both Cangzhou and Qionghai in the Hebei province of China?', 'answer': 'no', 'gold_titles': {'Cangzhou', 'Qionghai'}}) (input_keys={'question'}),
 Example({'question': 'Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?', 'answer': 'National Hockey League', 'gold_titles': {'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}}) (input_keys={'question'}),
 Example({'question': 'The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay Lightning of the National Hockey League (NHL)?', 'answer': 'Steve Yzerman', 'gold_titles': {'2006–07 Detroit Red Wings season', 'Steve Yzerman'}}) (input_keys={'question'}),
 Example({'question': 'What river is near the Crichton Collegiate Church?', 'answer': 'the River Tyne', 'gold_titles': {'Crichton Castle', 'Crichton Collegiate Church'}}) (input_keys={'que

In [5]:
agent = dspy.ReAct("question -> answer", tools=[dspy.Retrieve(k=1)])

In [ ]:
config = dict(num_threads=8, display_progress=True, display_table=5)
evaluate = Evaluate(devset=devset, metric=dspy.evaluate.answer_exact_match, **config)

evaluate(agent)

In [ ]:
lm.inspect_history(n=10)

In [7]:
config = dict(max_bootstrapped_demos=2, max_labeled_demos=0, num_candidate_programs=5, num_threads=8)
tp = BootstrapFewShotWithRandomSearch(metric=dspy.evaluate.answer_exact_match, **config)
optimized_react = tp.compile(agent, trainset=trainset, valset=valset)

  0%|          | 0/50 [00:00<?, ?it/s]

2024-06-05T16:05:32.199127Z [error    ] Error for example in dev set: 		 HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=120) [dspy.evaluate.evaluate] filename=evaluate.py lineno=147
2024-06-05T16:05:40.822703Z [error    ] Error for example in dev set: 		 HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=120) [dspy.evaluate.evaluate] filename=evaluate.py lineno=147


Average Metric: 4 / 21  (19.0):  42%|████▏     | 21/50 [09:08<07:17, 15.09s/it]

In [ ]:
optimized_react.save("optimized_react.json")

In [ ]:
loaded_optimized_react = dspy.ReAct("question -> answer", tools=[dspy.Retrieve(k=1)])
loaded_optimized_react.load("file.json")